<a href="https://colab.research.google.com/github/BrunoBustos96/DeteccionSomnolenciaCNN/blob/main/Parte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Adquisición y limpieza de Datos**

Función Eye Crooping para imágenes de cara completa.
Este notebook permitirá tomar una imagen de cara completa y extraer el ojo izquierdo y derecho de la imagen. Los resultados se guardan en carpetas de entrenamiento y de prueba para utilizarlos en el entrenamiento de los modelos de este proyecto.

Datos de Ojos Cerrados, citación: https://parnec.nuaa.edu.cn/_upload/tpl/02/db/731/template731/pages/xtan/ClosedEyeDatabases.html

Datos de Ojos Abiertos, citación: http://vis-www.cs.umass.edu/lfw/

Después de ejecutar este código, las imágenes producidas pasaron por un riguroso proceso de revisión para confirmar que eran efectivamente, ojos cerrados o abiertos. Las imágenes revisadas utilizadas para el entrenamiento del modelo se encuentran en 'new_closed_eyes.zip' y 'new_open_eyes.zip'.

# **Importar**

In [ ]:
#El modulo "face_recognition" es necesario para que pueda correr el presente Colab, debe instalarse 
# https://pypi.org/project/face-recognition/
!pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100.1 MB 27 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=f7c4c4ff440e80ba1dabda9bbb548f010a6c399da7202f370e15938f288bc4c2
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
!python setup.py install --no DLIB_USE_CUDA

python3: can't open file 'setup.py': [Errno 2] No such file or directory


In [ ]:
# OS nos permite usar funcionalidades dependientes del sistema operativo, lectura, escritura de archivos, manipulación de rutas.
import os
#PIL permite la edicion de imagenes desde Python
#Image provee funciones para cargar imagenes desde archivos
#ImageDraw provee funcionalidades de graficos en 2D en objetos de Image, creando nuevas imagenes, retoque y anotación de imagenes 
from PIL import Image, ImageDraw
#Se importa la libreria del modulo previamente instalado, el cual permite el reconocimiento de rostros en Python
import face_recognition

# **Montar la unidad para acceder a los archivos de la unidad**

In [ ]:
# # Descomnentar si funciona en Google Colab
# Montar Google Drive en tiempo de ejecución utilizando un código de autorización, escribir y leer archivos allí.
# from google.colab import drive
# drive.mount('/content/drive/')

# **Creación de la Función**

Estas funciones toman imágenes de rostros completos y recortan los ojos de esa imagen. A continuación, las almacena en la carpeta de datos "train" para utilizarlas en el entrenamiento y ajuste del modelo.

In [ ]:
#Funcion para recortar imagen de ojos abiertos  

def eye_cropper(folders):
    
    # Establecer contador (count) para guardar archivo de manera iterativa
    count = 0

    # Usar bucle FOR para recorrer todos los archivos de imagen 
    for folder in os.listdir(folders):
        for file in os.listdir(folders + '/' + folder):
          
            # Se utiliza la libreria face_recognition en las imagenes
            image = face_recognition.load_image_file(folders + '/' + folder + '/' + file)
          
            # Crear variable para determinar las coordenadas de los rasgos faciales 
            face_landmarks_list = face_recognition.face_landmarks(image)

            # Se crea un array para marcar la posición de las coordenadas de los ojos 
            # y añadir las coordenadas de los ojos que no hayan sido encontrados 
            eyes = []
            try:
                eyes.append(face_landmarks_list[0]['left_eye'])
                eyes.append(face_landmarks_list[0]['right_eye'])
            except:
                continue

            # Establecer las coordenadas X y Y maximas de los ojos
            for eye in eyes:
                x_max = max([coordinate[0] for coordinate in eye])
                x_min = min([coordinate[0] for coordinate in eye])
                y_max = max([coordinate[1] for coordinate in eye])
                y_min = min([coordinate[1] for coordinate in eye])

              # Establecer el rango de las coordenadas X y Y    
                x_range = x_max - x_min
                y_range = y_max - y_min
              
              # Para asegurar que el ojo entero es capturado, calcular las coordenadas de un cuadrado que tenga
              # 50% de brecha al eje mayor y hacerlo coincidir el rango menor con el rango mayor



                if x_range > y_range:
                    right = round(.5*x_range) + x_max
                    left = x_min - round(.5*x_range)
                    bottom = round(((right-left) - y_range))/2 + y_max
                    top = y_min - round(((right-left) - y_range))/2
                else:
                    bottom = round(.5*y_range) + y_max
                    top = y_min - round(.5*y_range)
                    right = round(((bottom-top) - x_range))/2 + x_max
                    left = x_min - round(((bottom-top) - x_range))/2
              
                # Guardar la imagen original como una variable
                im = Image.open(folders + '/' + folder + '/' + file)
                
                # Cortar la imagen original usando las coordenadas
                im = im.crop((left, top, right, bottom))
              
                # Redimensionar la imagen para agregar a nuestro modelo
                im = im.resize((80,80))
              
                # Guardar archivo a la carpeta de salida
                # Actualizar la carpeta donde se almacenan las imagenes recortadas de los ojos
                im.save('../data/train/open_eyes/eye_crop_open' + str(count) + '.jpg')
                #im.save('/content/drive/MyDrive/DSIR_2.8_Group_Project/Data/train/Open_Eyes/eye_crop_open' + str(count) + '.jpg')
              
                # Incrementar contador al guardar el archivo de manera iterativa
                count += 1
              
                # Imprimir contadaor cada 100 fotos guardadas para monitorear el proceso
                if count % 100 == 0:
                    print(count)
    
# Se llama a la funcion para cortar el rostro entero y de ojos abiertos
eye_cropper('../data/open_eyes_face_data')     

# Si se utiliza Google Colab, cambiar el directorio donde se almacenan las imagenes de rostro completo 
# eye_cropper('/content/drive/MyDrive/DSIR_2.8_Group_Project/Data/open_eyes_face_data')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [ ]:
#Funcion para recortar imagen de ojos cerrados  

def eye_cropper(folder):
    
    # Establecer contador (count) para guardar archivo de manera iterativa
    count = 0

    # Usar bucle FOR para recorrer todos los archivos de imagen 
    for file in os.listdir(folder):
        
        # Se utiliza la libreria face_recognition en las imagenes

        image = face_recognition.load_image_file(folder + '/' + file)
        
        # Crear variable para determinar las coordenadas de los rasgos faciales 

        face_landmarks_list = face_recognition.face_landmarks(image)
        
        # Se crea un array para marcar la posición de las coordenadas de los ojos 
        # y añadir las coordenadas de los ojos al array a menos que no hayan sido reconocidos por face_recognition 
        eyes = []
        try:
            eyes.append(face_landmarks_list[0]['left_eye'])
            eyes.append(face_landmarks_list[0]['right_eye'])
        except:
            continue

        # Establecer las coordenadas X y Y maximas de los ojos
        for eye in eyes:
            x_max = max([coordinate[0] for coordinate in eye])
            x_min = min([coordinate[0] for coordinate in eye])
            y_max = max([coordinate[1] for coordinate in eye])
            y_min = min([coordinate[1] for coordinate in eye])

        # Establecer el rango de las coordenadas X y Y    
            x_range = x_max - x_min
            y_range = y_max - y_min
            
            
            # Para asegurar que el ojo entero es capturado, calcular las coordenadas de un cuadrado que tenga
            # 50% de brecha al eje mayor y hacerlo coincidir el rango menor con el rango mayor de la brecha

            if x_range > y_range:
                right = round(.5*x_range) + x_max
                left = x_min - round(.5*x_range)
                bottom = round(((right-left) - y_range))/2 + y_max
                top = y_min - round(((right-left) - y_range))/2
            else:
                bottom = round(.5*y_range) + y_max
                top = y_min - round(.5*y_range)
                right = round(((bottom-top) - x_range))/2 + x_max
                left = x_min - round(((bottom-top) - x_range))/2
            
            # Guardar la imagen original como una variable
            im = Image.open(folder + '/' + file)
            
            # Cortar la imagen original usando las coordenadas de la brecha
            im = im.crop((left, top, right, bottom))
            
            # Redimensionar la imagen para agregar a nuestro modelo
            im = im.resize((80,80))
            
            # Guardar archivo a la carpeta de salida
            # Actualizar la carpeta donde se almacenan las imagenes recortadas de los ojos
            im.save('../data/train/closed_eyes/eye_crop_closed' + str(count) + '.jpg')

#             im.save('/content/drive/MyDrive/DSIR_2.8_Group_Project/Data/train/Closed_Eyes/eye_crop_closed' + str(count) + '.jpg')
            
            # Incrementar contador al guardar el archivo de manera iterativa
            count += 1
            
            # Imprimir contadaor cada 10 fotos guardadas para monitorear el proceso
            if count % 10 == 0:
                print(count)

# Se llama a la funcion para cortar el rostro entero y de ojos cerrados
eye_cropper('../data/closed_eyes_face_data')  

# Si se utiliza Google Colab, cambiar el directorio donde se almacenan las imagenes de rostro completo 
# eye_cropper('/content/drive/MyDrive/DSIR_2.8_Group_Project/Data/dataset_B_FacialImages_highResolution')

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
